In [22]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [23]:
 torch.manual_seed(42)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device} ")

Using device: cuda 


In [25]:
data_path = "../09-Hyperparameter-Tuning-Bayesian Search/fashion-mnist_train.csv"

import pandas as pd
df = pd.read_csv(data_path)

In [26]:
X = df.iloc[:, 1: ].values
y = df.iloc[:, 0].values

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## ***Build our own transformations***

In [28]:
from torchvision.transforms import transforms

custom_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean = [0.485, 0.456, 0.406],
            std = [0.229, 0.224, 0.225]
        )
    ]
)

In [29]:
from PIL import Image
import numpy as np

class CustomDataset(Dataset):

    def __init__(self, features, labels, transform):
        super().__init__()
        self.features = features
        self.label = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        # resize 28, 28
        image = self.features[index].reshape(28, 28)
        # change datatype np.uint8
        image = image.astype(np.uint8)

        # change B&W to collor
        image = np.stack([image]*3, axis=-1)
        
        # convert array to PIL image
        image = Image.fromarray(image)
        
        # apply transform
        image = self.transform(image)
        # print(image.shape)

        return image, torch.tensor(self.label[index], dtype=torch.long)

In [30]:
train_dataset = CustomDataset(X_train, y_train, transform=custom_transform)

In [31]:
test_dataset = CustomDataset(X_test, y_test, transform=custom_transform)

In [32]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)

## Fetch the pretrin model

In [33]:
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

/home/md-al-amin/miniconda3/envs/torch-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/md-al-amin/miniconda3/envs/torch-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [34]:
## Frize the pretrain parameter by required gurd is False

for param in vgg16.features.parameters():
    param.requires_grad=False

In [35]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [15]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [36]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

In [37]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [38]:
vgg16 = vgg16.to(device)

In [39]:
learning_rate = 0.0001
epochs = 10

In [42]:
criterion = nn.CrossEntropyLoss()
optimzer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [43]:
## traing loop

for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_loader:
        ## move dataset into the gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        ## forward
        output = vgg16(batch_features)

        ## loss
        loss = criterion(output, batch_labels)

        ## backward
        optimzer.zero_grad()
        loss.backward()

        ## update the weight
        optimzer.step()

        total_epoch_loss = total_epoch_loss + loss.item()

    avg_loss = total_epoch_loss/len(train_loader)
    print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1 , Loss: 0.3670301072448492
Epoch: 2 , Loss: 0.21841417237184943
Epoch: 3 , Loss: 0.1679442491264393
Epoch: 4 , Loss: 0.13248363182352235
Epoch: 5 , Loss: 0.10481540832016617
Epoch: 6 , Loss: 0.08372377750750941
Epoch: 7 , Loss: 0.06608273853835998
Epoch: 8 , Loss: 0.05476379886280241
Epoch: 9 , Loss: 0.04864088296199528
Epoch: 10 , Loss: 0.04174656290608012


In [44]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = vgg16(batch_features)
    # print(outputs[0])

    _, predicted = torch.max(outputs, 1)
    # print(predicted)

    total = total + batch_labels.shape[0]
    # print((predicted == batch_labels).sum().item())

    correct = correct + (predicted == batch_labels).sum().item()
    # break

print(correct/total)


0.9168333333333333
